In [ ]:
# Código 7.0:  Bibliotecas e programas %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

#                 rode sempre este código antes dos demais

import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels.api as sm  # saida mais detalhada que LogisticRegression do sklearn.linear_model
# sm.logit usa método de maxima verossimilhança (tradicional) para rodar a logística
#o LogisticRegression usa uma implementação de otimização que pode ser ajustado dependendo do parâmetro solver
from sklearn.metrics import roc_curve, auc,accuracy_score, precision_score, recall_score, f1_score,matthews_corrcoef,confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# Código 7.1: Criando o dataframe kim %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Primeiro baixe o arquivo Kimsport6 do github e salve em seu computador
# faça o upload do arquivo no colab
kim=pd.read_excel('/content/sample_data/Kimsport6.xlsx')
display(kim.head(3))

In [ ]:
# Código 7.2: Separação em duas amostras - treinamento e teste %%%%%%%%%%%%%%%%%


X= kim.drop('STATUS_MP', axis=1)
print(X.columns)
y= kim['STATUS_MP']
y=pd.DataFrame(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=18)

# Verificar os tamanhos dos conjuntos e as proporções de STATUS
print("Tamanho do conjunto de treinamento:", X_train.shape)
print("Tamanho do conjunto de teste:", X_test.shape)
frequencia_train = y_train['STATUS_MP'].value_counts(normalize=True,dropna=False).sort_index()
print(frequencia_train)
frequencia_test = y_test['STATUS_MP'].value_counts(normalize=True,dropna=False).sort_index()
print(frequencia_test)

In [ ]:
# Código 7.3:  Regressao Logística utilizando Backward Selection %%%%%%%%%%%%%%

X_train = sm.add_constant(X_train) # sm.logit necessita inclusao de beta(zero)

#rodando e selecionando as variáveis ==========================================
def backward_elimination(X_train, y_train, significance_level=0.15):
    # Ajustando o modelo de regressão logística
    model = sm.Logit(y_train, X_train)
    result = model.fit()

    # Mostrando o resumo  do modelo com todas as preditoras
    print("Resumo do modelo inicial:")
    print(result.summary())

    # Realizando a eliminação backward
    while max(result.pvalues) > significance_level:
        # Identificar o maior p-valor
        max_p_value = result.pvalues.idxmax()

        # Remover a variável com o maior p-valor
        print(f"\nRemovendo a variável: {max_p_value} com p-valor: {result.pvalues[max_p_value]}")
        X_train = X_train.drop(columns=[max_p_value])

        # Ajustando o modelo novamente com as variáveis restantes
        model = sm.Logit(y_train, X_train)
        result = model.fit()

    # Resumo final do modelo após a eliminação
    print("\nResumo final após backward elimination:")
    print(result.summary())

    return result, X_train
#cuidado: result só existe dentro da função. Nao é valor salvo
#===============================================================================
# Executando a eliminação backward com a amostra de treinamento
final_model, X_selected = backward_elimination(X_train, y_train)
# O modelo final está em final_model, e as variáveis selecionadas em X_selected
print(X_selected.columns)
#===============================================================================
# colocando colunas em X_test para ficar igual a X_selected
#(ou seja,, sem as vars elinminadas no Bward)
selected_columns = X_selected.columns[1:]
# Filter X_test using selected_columns
X_test = X_test[selected_columns]
X_test = sm.add_constant(X_test)  # Add constant back to X_test
#===============================================================================
#Previsões das probabilidades de mau pagador
y_pred_prob = final_model.predict(X_test)
print("\n probs. prevista",y_pred_prob.head(3))
#===============================================================================
# gerando um arquivo com todas as colunas de X_test usadas no modelo
# e as probabilidade prevista de ser mau pagador y_pred_prob ==> X_FINAL
X_FINAL=X_test.copy()
X_FINAL['y_pred_prob'] = y_pred_prob
# Exibindo o DataFrame atualizado
print(X_FINAL.head(3))
#===============================================================================
# Calcular a curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
# Calcular a AUC (Área sob a Curva)
roc_auc = auc(fpr, tpr)
# Plotar a curva ROC
plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label=f'Curva ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.show()
# Exibir a AUC
print(f'AUC: {roc_auc:.2f}')
#===============================================================================
# Avaliação do modelo para um ponto de corte PC
PC=0.5
y_pred = final_model.predict(X_test)
y_pred = (y_pred_prob >= PC).astype(int)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
MCC=matthews_corrcoef(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

frequencia_test = y_test['STATUS_MP'].value_counts(dropna=False).sort_index()
print("frequencia de bons (0) e maus (1) pagadores",frequencia_test)

print("\n Acurácia:", accuracy)
print("Precisão:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Mathews correlation coeficient:", f1)
print("Matriz de Classificação:")
print(conf_matrix)

In [ ]:
#  Código 7.4: seleção de variáveis utilizando AIC - Akaike infrormation criteria
#   apenas uma ilustração desta forma de seleção de variáveis, nao está no texto
#   programa seleciona a variavel a remover usando p-value;
#   depois calcula AIC e vê se melhorou (redução de AIC)
# (o software R tem um algoritmo melhor para usar AIC)

#repetindo separação pois arquivos foram modificados no código 7.3 acima
X= kim.drop('STATUS_MP', axis=1)
print(X.columns)
y= kim['STATUS_MP']
y=pd.DataFrame(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=18)

# Adicionar constante (para manter o intercepto no modelo)
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

# Função para AIC backward selection mantendo o intercepto
def backward_selection(X, y):
    model = sm.Logit(y, X).fit(disp=1)  # Ajusta modelo inicial
    print(f"Modelo inicial - AIC: {model.aic:.4f} - Variáveis: {list(X.columns)}\n")
    print(model.summary())
    while True:
        aic = model.aic
        pvalues = model.pvalues.drop("const")  # Remove apenas a constante da análise
        if pvalues.empty:
            break
        worst_pval = pvalues.max()  # Variável com maior p-valor
        if worst_pval > 0.15:  # Limite de significância
            worst_feature = pvalues.idxmax()
            X = X.drop(columns=[worst_feature])  # Remove variável
            model = sm.Logit(y, X).fit(disp=0)  # Reajusta modelo
            print(f"Removendo '{worst_feature}' - Novo AIC: {model.aic:.4f} - Variáveis: {list(X.columns)}")
            if model.aic >= aic:  # Se AIC não melhorar, para
                break
        else:
            break

    print("\nSeleção final concluída!\n")
    return model, X

# Aplicar seleção de variáveis
modelo_final, X_train_final = backward_selection(X_train_const, y_train)
print("\n", modelo_final.summary())

# Ajustar modelo final e fazer previsões no conjunto de teste
X_test_final = X_test_const[X_train_final.columns]  # Manter mesmas colunas

PC=0.5
y_pred_probAIC = modelo_final.predict(X_test_final) #com AIC
y_pred_classAIC = (y_pred_prob2 >= PC).astype(int)  # Converter para classes binárias


In [ ]:
#  Código 7.5: gerando um arquivo com todas as colunas de X_test e as probabilidade previstas ==> kim_final_reglog

kim_final_reglog=X_test.copy()
kim_final_reglog["status_mp"]=y_test
kim_final_reglog['y_pred_prob'] = y_pred_prob

# Exibindo o DataFrame atualizado
print(kim_final_reglog.head(3))

!pip install openpyxl > /dev/null
kim_final_reglog.to_excel('kim_final_reglog.xlsx', index=False)
from google.colab import files
files.download('kim_final_reglog.xlsx')